# IbPy and Interactive Brokers Features Demonstration
Tested on Python 2

## Learning Outcomes
At the end of this simple workshop, you will be able to 
1. Extract Account and Portfolio Informatioin
2. Placing Orders
3. Request Market Data
4. Obtain Historical Data
5. Access Market Depth Informaation
6. Download Real Time Bars
7. Extract Executions Information, including commission report

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
import time
from datetime import datetime
from IBWrapper import IBWrapper, contract
from ib.ext.EClientSocket import EClientSocket
from ib.ext.ScannerSubscription import ScannerSubscription

In the example to follow, **`callback`** is our `IBWrapper` instantiated. We **receive** information via `callback`.


In the example to follow, **`tws`** is our `EClientSocket` instantiated. We **request** information via `tws`

In [3]:
accountName = "DU254946"
callback = IBWrapper()             # Instantiate IBWrapper. callback 
tws = EClientSocket(callback)      # Instantiate EClientSocket and return data to callback
host = ""
port = 4001
clientId = 5555

In [4]:
tws.eConnect(host, port, clientId) # Connect to TWS

Server Version: 76
TWS Time at connection:20160512 15:46:21 SGT


In [5]:
tws.setServerLogLevel(5)           # Set error output to verbose

In [6]:
create = contract()                # Instantiate contract class
callback.initiate_variables()

Note how the work flow goes. We send a request via the prefix **tws.** followed by the request for the specific type of data after the dot for the information we are interested in. 

For example, we would like to get an update on account time, which required us calling `reqAccountUpdates`. 

We request for info by calling **`tws.reqAccountUpdates`** and the data will be returned via our callback function. In this case **`callback.update_AccountTime`**

*****

# Account and Portfolio
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Account Updates
   * Account Value
   * Portfolio
   * Account Time
2. Account Summary
3. Positions

### Summary of Account and Portfolio 

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqAccountUpdates | updateAccountValue | self.update_AccountValue |
| | updatePortfolio | self.update_Portfolio |
| | updateAccountTime | self.update_AccountTime |
| reqAccountSummary | accountSummary | self.account_Summary |
| reqPositions | position | self.update_Position |

### Sending Account Updates Request
`reqAccountUpdates`

In [7]:
tws.reqAccountUpdates(1, accountName)

#### Obtaining Account Value
`self.update_AccountValue`

In [8]:
pd.DataFrame(callback.update_AccountValue, 
            columns = ['key', 'value', 'currency', 'accountName'])[:3]

,key,value,currency,accountName
0,AccountType,INDIVIDUAL,None,DU254946
1,AccountCode,DU254946,None,DU254946
2,AccountReady,true,None,DU254946


#### Obtaining Portfolio Value
`self.update_Portfolio`

In [9]:
pd.DataFrame(callback.update_Portfolio, 
             columns=['Contract ID','Currency',
                      'Expiry','Include Expired',
                      'Local Symbol','Multiplier',
                      'Primary Exchange','Right',
                      'Security Type','Strike',
                      'Symbol','Trading Class',
                      'Position','Market Price','Market Value',
                      'Average Cost', 'Unrealised PnL', 'Realised PnL', 
                      'Account Name'])[:3]

,Contract ID,Currency,Expiry,Include Expired,Local Symbol,Multiplier,Primary Exchange,Right,Security Type,Strike,Symbol,Trading Class,Position,Market Price,Market Value,Average Cost,Unrealised PnL,Realised PnL,Account Name
0,215465490,USD,20170317,False,ESH7,50,GLOBEX,0,FUT,0.0,ES,ES,5,2039.325000,509831.25,102324.530000,-1791.40,0.0,DU254946
1,12087792,USD,None,False,EUR.USD,None,IDEALPRO,0,CASH,0.0,EUR,EUR.USD,670000,1.141020,764483.44,1.090984,33524.48,0.0,DU254946
2,15016059,JPY,None,False,USD.JPY,None,IDEALPRO,0,CASH,0.0,USD,USD.JPY,100000,108.861999,10886199.95,118.603250,-974125.05,0.0,DU254946


#### Obtaining Account Time
`self.update_AccountTime`

In [10]:
callback.update_AccountTime

'15:46'

### Sending Account Summary Request
`reqAccountSummary`

This function call can only be made when connected to a Financial Advisor (FA) account. Another way to look at this is that if you have more than one account, use this function.

In [11]:
tws.reqAccountSummary(2,"All","NetLiquidation")

#### Obtaining Account Summary
`self.account_Summary`

In [12]:
pd.DataFrame(callback.account_Summary, 
             columns = ['Request_ID','Account','Tag','Value','Curency'])[:2]

,Request_ID,Account,Tag,Value,Curency
0,2,DI246990,NetLiquidation,1032397.94,USD
1,2,DU254946,NetLiquidation,803830.58,USD


### Sending Position Request
`reqPositions`

This function call request all positions for all accounts. This is more suitable for Financial Advisor. In the following example, I used pandas selection criteria to disply a specific account position.

In [13]:
tws.reqPositions()

#### Obtaining Position
`self.update_Position`

In [14]:
dat = pd.DataFrame(callback.update_Position, 
                   columns=['Account','Contract ID','Currency','Exchange','Expiry',
                            'Include Expired','Local Symbol','Multiplier','Right',
                            'Security Type','Strike','Symbol','Trading Class',
                            'Position','Average Cost'])
dat[dat["Account"] == accountName]

,Account,Contract ID,Currency,Exchange,Expiry,Include Expired,Local Symbol,Multiplier,Right,Security Type,Strike,Symbol,Trading Class,Position,Average Cost
67,DU254946,215465490,USD,None,20170317,False,ESH7,50,None,FUT,0.0,ES,ES,5,102324.530000
68,DU254946,12087792,USD,None,None,False,EUR.USD,None,None,CASH,0.0,EUR,EUR.USD,670000,1.090984
69,DU254946,15016059,JPY,None,None,False,USD.JPY,None,None,CASH,0.0,USD,USD.JPY,100000,118.603250


*****

# Orders
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Open Order
2. Next Valid ID
3. Order Status

### Summary of Orders

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqIds | nextValidId | self.next_ValidId |
| placeOrder | orderStatus | self.order_Status |
| cancelOrder | | |
| reqOpenOrders & reqAllOpenOrders | openOrder | self.open_Order |
| | orderStatus | self.order_Status |
| reqGlobalCancel | | |

#### Demo - Stock Purchase
* ** Request Next Valid Id**. `reqIds`
* ** Using Create**. `create`

In [15]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract("GOOG", "STK", "SMART", "USD")
order_info = create.create_order(accountName, "MKT", 100, "BUY")

#### Placing an Order
`placeOrder`

In [16]:
tws.placeOrder(order_id, contract_info, order_info)

#### Checking Order Status
`self.order_Status`

In [17]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

[2, 399, 'Order Message:\nBUY 100 GOOG NASDAQ.NMS\nWarning: your order will not be placed at the exchange until 2016-05-12 09:30:00 US/Eastern']


,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,2,PreSubmitted,0,100,0.0,2119447479,0,0.0,5555,None


#### Checking on Open Order
`self.open_Order`

In [18]:
callback.open_Order[:1]

[(2,
  <ib.ext.OrderState.OrderState at 0x8a98940>)]

#### Cancelling Open Order
`cancelOrder`

In [19]:
tws.cancelOrder(order_id)

#### Demo
* ** Request Next Valid Id**. `reqIds`
* ** Using Create**. `create`
* ** Placing an Order to purchase Futures**. `placeOrder`

In [20]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract(symbol = "ES", secType = "FUT", 
                                       exchange = "GLOBEX", currency = "USD", 
                                       right = None, strike = None,
                                       expiry = "201703", multiplier=None,
                                       tradingClass=None)
order_info = create.create_order(accountName, "MKT", 1, "BUY")
tws.placeOrder(order_id, contract_info, order_info)

[2, 202, 'Order Canceled - reason:']


#### Checking Order Status

In [21]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

[2, 104, "Can't modify a filled order"]


,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,2,PreSubmitted,0,100,0.0,2119447479,0,0.0,5555,None
1,2,PendingCancel,0,100,0.0,2119447479,0,0.0,5555,None
2,2,Cancelled,0,100,0.0,2119447479,0,0.0,5555,None


IB provided two more methods:
* `reqOpenOrders()` to request any open orders that were placed from this API client.
* `reqAllOpenOrders()` to request all open orders that were placed from all API clients linked to one TWS and also from the TWS.

Each open order will be fed back through `openOrder()` and `orderStatus()` methods.

Finally, use `reqGlobalCancel()` to cancel all open orders globally.

*****

# Market Data
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Market Data
   * Tick Price
   * Tick Size
2. Cancel Market Data
3. Calculate Implied Volatility
4. Calculate Option Price

### Summary of Market Data

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqMktData | tickPrice  | self.tick_Price |
|  | tickSize | self.tick_Size |
|  | tickOptionComputation  | self.tick_OptionComputation |
|  | tickGeneric | self.tick_Generic |
|  | tickString | self.tick_String |
|  | tickEFP  | self.tick_EFP |
|  | tickSnapshotEnd | self.tickSnapshotEnd_flag |
| cancelMktData | | |
| calculateImpliedVolatility | tickOptionComputation  | self.tick_OptionComputation |
| cancelcalculateImpliedVolatility | | |
| calculateOptionPrice  | tickOptionComputation  | self.tick_OptionComputation |
| cancelCalculateOptionPrice | | |
| reqMktDataType | marketDataType | self.market_DataType |

The method `reqMktDataType` allows you to toggle between receiving real-time or frozen market data.

#### Requesting Market Data
`reqMktData`

In [22]:
contract_info = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')
tickedId = 1002
tws.reqMktData(tickedId, contract_info, "", False)

[-1, 2119, 'Market data farm is connecting:cashfarm']
[-1, 2104, 'Market data farm connection is OK:cashfarm']


#### Receiving Tick Price
`self.tick_Price`

In [23]:
tick_data = pd.DataFrame(callback.tick_Price, 
                         columns = ['tickerId', 'field', 'price', 'canAutoExecute'])
tick_type = {0 : "BID SIZE",
             1 : "BID PRICE",
             2 : "ASK PRICE",
             3 : "ASK SIZE",
             4 : "LAST PRICE",
             5 : "LAST SIZE",
             6 : "HIGH",
             7 : "LOW",
             8 : "VOLUME",
             9 : "CLOSE PRICE",
             10 : "BID OPTION COMPUTATION",
             11 : "ASK OPTION COMPUTATION",
             12 : "LAST OPTION COMPUTATION",
             13 : "MODEL OPTION COMPUTATION",
             14 : "OPEN_TICK",
             15 : "LOW 13 WEEK",
             16 : "HIGH 13 WEEK",
             17 : "LOW 26 WEEK",
             18 : "HIGH 26 WEEK",
             19 : "LOW 52 WEEK",
             20 : "HIGH 52 WEEK",
             21 : "AVG VOLUME",
             22 : "OPEN INTEREST",
             23 : "OPTION HISTORICAL VOL",
             24 : "OPTION IMPLIED VOL",
             27 : "OPTION CALL OPEN INTEREST",
             28 : "OPTION PUT OPEN INTEREST",
             29 : "OPTION CALL VOLUME"}
tick_data["Type"] = tick_data["field"].map(tick_type)
tick_data[-10:]

,tickerId,field,price,canAutoExecute,Type
0,1002,6,1.14295,0,HIGH
1,1002,7,1.14050,0,LOW
2,1002,9,1.14270,0,CLOSE PRICE
3,1002,1,1.14105,1,BID PRICE
4,1002,2,1.14110,1,ASK PRICE


#### Receiving Tick Size
`self.tick_Size`

In [24]:
tick_data = pd.DataFrame(callback.tick_Size, 
                         columns = ["tickerId", "field", "size"])
tick_data["Type"] = tick_data["field"].map(tick_type)
tick_data[-10:]

,tickerId,field,size,Type
49,1002,0,15605000,BID SIZE
50,1002,0,14605000,BID SIZE
51,1002,3,1000000,ASK SIZE
52,1002,0,17605000,BID SIZE
53,1002,0,14605000,BID SIZE
54,1002,0,16605000,BID SIZE
55,1002,0,17605000,BID SIZE
56,1002,3,18350000,ASK SIZE
57,1002,3,18350000,ASK SIZE
58,1002,0,18605000,BID SIZE


#### Calculate Implied Volatility
`calculateImpliedVolatility`

In [25]:
contract_info = create.create_contract(symbol='NFLX 160819C00095000',
                                       secType='OPT', exchange='SMART', 
                                       currency='USD',
                                       right='CALL', 
                                       strike='95', 
                                       expiry='20160819',
                                       multiplier=100, 
                                       tradingClass="NFLX")
tws.calculateImpliedVolatility(tickedId, 
                               contract_info, 
                               5.89, 
                               89.91)

In [27]:
pd.DataFrame(callback.tick_OptionComputation,
             columns=["tickerId", "field", "impliedVol", "delta",
                      "optPrice", "pvDividend", "gamma", "vega",
                      "theta", "undPrice"])

,tickerId,field,impliedVol,delta,optPrice,pvDividend,gamma,vega,theta,undPrice
0,1002,53,0.425206,2147483647,5.89,2147483647,2147483647,2147483647,2147483647,89.91


#### Calculate Option Price
`tick_OptionComputation`

In [28]:
tws.calculateOptionPrice(tickedId, 
                         contract_info, 
                         0.84, 
                         89.91)

In [29]:
pd.DataFrame(callback.tick_OptionComputation,
             columns=["tickerId", "field", "impliedVol", "delta",
                      "optPrice", "pvDividend", "gamma", "vega",
                      "theta", "undPrice"])

[-1, 2119, 'Market data farm is connecting:usopt']


,tickerId,field,impliedVol,delta,optPrice,pvDividend,gamma,vega,theta,undPrice
0,1002,53,0.425206,2147483647,5.89,2147483647,2147483647,2147483647,2147483647,89.91


#### Cancelling Market Data Stream
`cancelMktData`

In [30]:
tws.cancelMktData(tickedId)

[-1, 2104, 'Market data farm connection is OK:usopt']


*****

# Historical Data
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Historical Data

### Summary of Historical Data

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqHistoricalData | historicalData  | self.historical_Data |

In [31]:
#contract_Details = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')
contract_Details = create.create_contract('AAPL', 'STK', 'SMART', 'USD')

[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.cashfarm']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.cashfarm']


In [32]:
data_endtime = datetime.now().strftime("%Y%m%d %H:%M:%S")

#### Requesting Historical Data
`reqHistoricalData`

In [33]:
tickerId = 9002
tws.reqHistoricalData(tickerId, 
                      contract_Details, 
                      data_endtime,
                      "1 M", 
                      "1 day", 
                      "BID", 
                      0, 
                      1)

In [35]:
data= pd.DataFrame(callback.historical_Data, 
                   columns = ["reqId", "date", "open",
                              "high", "low", "close", 
                              "volume", "count", "WAP", 
                              "hasGaps"])
data[-10:]

,reqId,date,open,high,low,close,volume,count,WAP,hasGaps
13,9002,20160429,94.45,94.95,92.51,93.60,-1,-1,-1.0,False
14,9002,20160502,93.75,94.30,92.40,93.81,-1,-1,-1.0,False
15,9002,20160503,93.50,95.73,93.01,95.12,-1,-1,-1.0,False
16,9002,20160504,95.23,95.88,93.82,94.36,-1,-1,-1.0,False
17,9002,20160505,1.00,94.37,1.00,93.12,-1,-1,-1.0,False
18,9002,20160506,89.80,93.48,89.80,92.67,-1,-1,-1.0,False
19,9002,20160509,89.80,93.76,89.80,92.61,-1,-1,-1.0,False
20,9002,20160510,89.80,93.57,89.80,93.26,-1,-1,-1.0,False
21,9002,20160511,92.50,93.56,92.25,92.34,-1,-1,-1.0,False
22,9002,finished-20160412 15:43:47-20160512 15:43:47,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.0,False


*****

# Market Depth
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Market Depth

### Summary of Market Depth

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqMktDepth           | updateMktDepth          | self.update_MktDepth |

In [36]:
contract_info = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')

In [37]:
tickerId = 7000
tws.reqMktDepth(tickerId, contract_info, 5)

In [38]:
operation_type = {0 : "Insert",
                  1 : "Update",
                  2 : "Delete",}
side_type = {0 : "Ask",
             1 : "Bid"}

[-1, 2119, 'Market data farm is connecting:cashfarm']


In [39]:
data_mktdepth = pd.DataFrame(callback.update_MktDepth,
                             columns = ["tickerId", "position", 
                                        "operation", "side", 
                                        "price", "size"])
data_mktdepth["operation_type"] = data_mktdepth["operation"].map(operation_type)
data_mktdepth["side_type"] = data_mktdepth["side"].map(side_type)
data_mktdepth[-10:]

[-1, 2104, 'Market data farm connection is OK:cashfarm']


,tickerId,position,operation,side,price,size,operation_type,side_type
70,7000,3,1,0,1.14120,1040000,Update,Ask
71,7000,0,1,1,1.14095,17605000,Update,Bid
72,7000,1,1,0,1.14105,17175000,Update,Ask
73,7000,0,1,1,1.14095,17430000,Update,Bid
74,7000,0,1,1,1.14095,16430000,Update,Bid
75,7000,1,1,0,1.14105,16175000,Update,Ask
76,7000,0,1,1,1.14095,17430000,Update,Bid
77,7000,1,1,0,1.14105,17175000,Update,Ask
78,7000,0,1,1,1.14095,16430000,Update,Bid
79,7000,1,1,0,1.14105,16175000,Update,Ask


*****

# Real Time Bars
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Real Time Bars

Note:
* **barSize**. Only 5 sec bars are supported. 
* **whatToShow**:
   * TRADES
   * BID
   * ASK
   * MIDPOINT
* **useRTH**:
   * 0 = all data
   * 1 = only data within **R**egular **T**rading **H**ours

### Summary of Real Time Bars

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqRealTimeBars           | realtimeBar          | self.real_timeBar |

In [40]:
contract_Details = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')

In [41]:
tickerId = 10000
tws.reqRealTimeBars(tickerId, 
                    contract_Details, 
                    5, 
                    "MIDPOINT", 
                    0)

In [43]:
pd.DataFrame(callback.real_timeBar, 
             columns = ["reqId", "time", "open", "high", "low", "close", "volume", "wap", "count"])

,reqId,time,open,high,low,close,volume,wap,count
0,10000,1463039280,1.140975,1.141025,1.140975,1.140975,-1,-1.0,-1


*****

# Executions
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Executions

### Summary of Executions

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqExecutions           | execDetails          | self.exec_Details_reqId |
| | | self.exec_Details_contract |
| | | self.exec_Details_execution |
| | execDetailsEnd |self.exec_DetailsEnd_flag |
| | commissionReport | self.commission_Report |

In [44]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract(symbol = "ES", secType = "FUT", 
                                       exchange = "GLOBEX", currency = "USD", 
                                       right = None, strike = None,
                                       expiry = "201703", multiplier=None,
                                       tradingClass=None)
order_info = create.create_order(accountName, "MKT", 1, "BUY")
tws.placeOrder(order_id, contract_info, order_info)

In [45]:
tws.reqExecutions(3050, create.exec_filter(clientId, accountName, contract_info))

In [46]:
callback.exec_Details_reqId

3050

In [47]:
callback.exec_Details_contract.__dict__

{'m_conId': 215465490,
 'm_currency': 'USD',
 'm_exchange': 'GLOBEX',
 'm_expiry': '20170317',
 'm_includeExpired': False,
 'm_localSymbol': 'ESH7',
 'm_multiplier': '50',
 'm_right': None,
 'm_secType': 'FUT',
 'm_strike': 0.0,
 'm_symbol': 'ES',
 'm_tradingClass': 'ES'}

In [48]:
callback.exec_Details_execution.__dict__

{'m_acctNumber': 'DU254946',
 'm_avgPrice': 2045.5,
 'm_clientId': 5555,
 'm_cumQty': 1,
 'm_evMultiplier': 0,
 'm_evRule': None,
 'm_exchange': 'GLOBEX',
 'm_execId': '0001f4e5.5733bba7.01.01',
 'm_liquidation': 0,
 'm_orderId': 4,
 'm_orderRef': None,
 'm_permId': 2119447480,
 'm_price': 2045.5,
 'm_shares': 1,
 'm_side': 'BOT',
 'm_time': '20160512  15:48:14'}

In [49]:
pd.DataFrame(callback.commission_Report.__dict__, index=[0])

,m_commission,m_currency,m_execId,m_realizedPNL,m_yield,m_yieldRedemptionDate
0,2.03,USD,0001f4e5.5733bba7.01.01,1.797693e+308,1.797693e+308,0


In [50]:
tws.eDisconnect()